In [1]:
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True, reset=True, n_consumers=4,
                           create_producer=True)
client = node.login(email="info@openmined.org", password="changethis")

INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
# Question 1: What type of container do we want for data already on the server?

obj_1 = sy.ActionObject.from_obj(1)
ptr_1 = obj_1.send(client)

obj_2 = sy.ActionObject.from_obj(2)
ptr_2 = obj_2.send(client)

obj_3 = sy.ActionObject.from_obj(3)
ptr_3 = obj_3.send(client)

# Option 1: ActionObjects inside ActionObjects
# 
# Pros: very versatile, could work with data from other domains out of the box
# Cons: might not feel intuitive to the user, will need to change the way we work with
#       ActionObjects in a lot of different places in the codebase
list = sy.ActionObject.from_obj([ptr_1, ptr_2, ptr_3])
list_ptr = list.send(client)

# Option 2: Create new ActionObjects from the same data
# Will require us to do some value based verification on different objects
# 
# Pros: Easier abstraction for the user
# Cons: Value based verification sounds like an attack vector
#       as it can provide a free Oracle to an attacker
# list = sy.ActionObject.from_pointer_list([ptr_1, ptr_2, ptr_3]) # on the server will do ActionObject.from_obj([1,2,3])
# list_ptr = list.send(client)
 

In [4]:
from syft.service.policy.policy import OutputPolicyExecuteCount

# Question 2: What should the UX be for ExecuteOncePerCombination?
# 
# Right now I have worked on using the first option from the previous question
# and using on the fly created lists. We can break this question into more specific ones:
#
# Sub-Question 1: What should we pass for each argument? Should the list be already on the server?
#                 Or can it be defined by the data scientist? 
#                 Could it be made of data outside the domain?
#
# Sub-Question 2: Will anything change if instead of data we talk about files?
#                 The final use case actually will iterate for SyftFiles, so can this affect the UX?
#

@sy.syft_function(input_policy=sy.ExecuteOncePerCombination(
                                    x=[ptr_1, ptr_2, ptr_3],
                                    y=[ptr_1, ptr_2, ptr_3],
                                    z=[ptr_1, ptr_2, ptr_3],
                                ),
                  output_policy=OutputPolicyExecuteCount(limit=27))
def func(x, y, z):
    import time
    time.sleep(5)
    return x, y, z

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
request = client.code.submit(func)


In [6]:
@sy.syft_function_single_use(list=list_ptr)
def main_func(domain, list):
    jobs = []
    print("start")
    domain.init_checkpoint(27)
    for x in list:
        for y in list:
            for z in list:
                print(x,y,z)
                domain.checkpoint()
                batch_job = domain.launch_job(func, x=x, y=y, z=z)
                jobs.append(batch_job)
                    
    print("done")
    
    return None

SyftSuccess: Syft function 'main_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
client.code.request_code_execution(main_func)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request 6b5b901d129b415cb535cfedaa0efe08 changes applied

In [8]:
job = client.code.main_func(list=list_ptr, blocking=False)

In [9]:
job

```python
class Job:
    id: UID = 2125ce96810243e092fbadb4b836ffc1
    status: JobStatus.CREATED
    has_parent: False
    result: None
    logs:

0 
    
```

In [10]:
from datetime import datetime
str(datetime.now())[:-7]

'2023-10-12 16:35:22'

In [17]:
jobs = client.jobs
jobs

In [12]:
from syft.node.node import AuthNodeContextRegistry

In [13]:
AuthNodeContextRegistry.__node_context_registry__

OrderedDict([('448cb501cca248c7b9845081ed40744c-47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323',
              syft.service.context.AuthedServiceContext),
             ('448cb501cca248c7b9845081ed40744c-4a3ade8a758653bbd563e914bcca9d295403103b953f8d3eb52cdba4a4cfc3f8',
              syft.service.context.AuthedServiceContext)])

In [14]:
# jobs[2].parent_job_id

In [15]:
job_tree = {}
for job in jobs:
    if job.parent_job_id in job_tree:
        job_tree[job.parent_job_id].append(job)
    else:
        job_tree[job.parent_job_id] = [job]

In [ ]:
def recursive_print(parent_job, tab_space = 0):
    lines = "─"
    if parent_job.id in job_tree:
        for job in job_tree[parent_job.id]:
            print(f"├─{lines * 2}",  job.id)
            recursive_print(job, tab_space=tab_space+2)

for job in jobs:
    if not job.has_parent:
        print("├─", job.id)
        recursive_print(job, tab_space=2)
        

├─ 2125ce96810243e092fbadb4b836ffc1


NEW QUEUE CALL: 2125ce96810243e092fbadb4b836ffc1 None
FUNCTION LOG ((<UID: 2125ce96810243e092fbadb4b836ffc1>, 100)): start
FUNCTION LOG ((<UID: 2125ce96810243e092fbadb4b836ffc1>, 100)): 1 1 1


LAUNCHING JOB func


FUNCTION LOG ((<UID: 2125ce96810243e092fbadb4b836ffc1>, 100)): 1 1 2


LAUNCHING JOB func


...

FUNCTION LOG ((<UID: 2125ce96810243e092fbadb4b836ffc1>, 100)): 3 3 2
NEW QUEUE CALL: d434214778874f14a38b9d1961b1b68f None
NEW QUEUE CALL: fdf9961fc1484fbf947fbfb5820a9f40 d434214778874f14a38b9d1961b1b68f


LAUNCHING JOB func


FUNCTION LOG ((<UID: 2125ce96810243e092fbadb4b836ffc1>, 100)): 3 3 3


LAUNCHING JOB func


NEW QUEUE CALL: cee2b687c51b4a36b246ccbc2836bc6c None
FUNCTION LOG ((<UID: 2125ce96810243e092fbadb4b836ffc1>, 100)): done
NEW QUEUE CALL: 58eec101c25649e9af0ab3623c56f331 None
NEW QUEUE CALL: 71f7d8d0e78f45faa3ffa6d314986cce None
NEW QUEUE CALL: 71ef707b184945bcbc116c5f5d843b25 None
NEW QUEUE CALL: 468953d6510d46feb0eec8ccbfe61cbe None
NEW QUEUE CALL: ca9ca19d6ebd49c7b5d61badc7690c6e None
NEW QUEUE CALL: b54613952d534b3290b56325eec7654b None
NEW QUEUE CALL: 02525f8847104cde98d7186946efb609 None
NEW QUEUE CALL: 061711b42c7846a2a0ed02fc08706c22 None
NEW QUEUE CALL: c90b8703cc834b48944ceaa2508d31fc None
NEW QUEUE CALL: fa441334d5b44b1da80bef75aece9a00 None
NEW QUEUE CALL: 775338749fa741868ac757a797c21834 None
NEW QUEUE CALL: 24e65ed4a5be4dd2a93698c82ac674e0 None
NEW QUEUE CALL: 22e42b168f5f4f739f640c540937ffb0 None
NEW QUEUE CALL: efd82c8de15c4833bfc0e9e4fd43b611 None
NEW QUEUE CALL: 844082d1588f459b861314cf2d67433b None
NEW QUEUE CALL: e67d978828c5437da24a1dede44a45cf None
NEW QUEUE CALL